In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
import os

import numpy as np
import pandas as pd
import scipy.sparse as sp

from lightfm import LightFM

/Users/liubovkupriianova/opt/anaconda3/envs/py36/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. Only a single thread will be used.')  # NOQA


In [3]:
!mkdir models

In [46]:
train_target = pd.read_pickle('dataset_v2.uu')

In [43]:
train = pd.read_pickle('user_prod.pkl')

In [51]:
train = pd.DataFrame(train)

In [56]:
items_feats = pd.read_pickle('prod.pkl')

In [53]:
train.columns = ['all_prods']

In [54]:
train_target = train_target.merge(train, on='user_id')

In [58]:
train = train_target

In [59]:
train.head()

,user_id,features,target_ids,all_prods
0,260,"[74.0, 51.0, 90.0, 13487.0, 66893.0, 5469736.0...","[66893, 158, 709]","[66893, 9496, 64615, 5469736, 13487, 10163, 0]"
1,353,"[90.0, 35.0, 9.0, 105764.0, 87886.0, 69372.0, ...","[2321, 2322, 407, 478, 80460, 69669]","[7071972, 97945, 478, 20269, 4443, 584, 57342,..."
2,400,"[84.0, 35.0, 20.0, 3817507.0, 0.0, 94333.0, 11...","[61053, 981, 177, 20286, 20289, 10198]","[94333, 5469728, 3817507, 45459, 407, 61053, 3..."
3,525,"[84.0, 90.0, 20.0, 709.0, 166.0, 55133.0, 75.6...","[709, 25645, 57174]","[709, 7397, 55133, 3817484, 166]"
4,576,"[20.0, 9.0, 119.0, 177.0, 2600.0, 10757.0, 103...","[177, 102]","[2600, 177, 7064842, 60105, 52657, 0, 4443, 10..."


In [60]:
train['num_targets'] = train['all_prods'].apply(lambda x: len(x))

In [61]:
train.head()

,user_id,features,target_ids,all_prods,num_targets
0,260,"[74.0, 51.0, 90.0, 13487.0, 66893.0, 5469736.0...","[66893, 158, 709]","[66893, 9496, 64615, 5469736, 13487, 10163, 0]",7
1,353,"[90.0, 35.0, 9.0, 105764.0, 87886.0, 69372.0, ...","[2321, 2322, 407, 478, 80460, 69669]","[7071972, 97945, 478, 20269, 4443, 584, 57342,...",52
2,400,"[84.0, 35.0, 20.0, 3817507.0, 0.0, 94333.0, 11...","[61053, 981, 177, 20286, 20289, 10198]","[94333, 5469728, 3817507, 45459, 407, 61053, 3...",31
3,525,"[84.0, 90.0, 20.0, 709.0, 166.0, 55133.0, 75.6...","[709, 25645, 57174]","[709, 7397, 55133, 3817484, 166]",5
4,576,"[20.0, 9.0, 119.0, 177.0, 2600.0, 10757.0, 103...","[177, 102]","[2600, 177, 7064842, 60105, 52657, 0, 4443, 10...",29


In [62]:
#train['user_ids'] = train['user_id'].apply(lambda x: x.index)
#list(train['num_targets'])

In [63]:
user_ids = np.array(train.user_id.repeat(list(train['num_targets'])))

In [64]:
user_ids

array([    260,     260,     260, ..., 1131904, 1131904, 1131904])

In [65]:
train.head()

,user_id,features,target_ids,all_prods,num_targets
0,260,"[74.0, 51.0, 90.0, 13487.0, 66893.0, 5469736.0...","[66893, 158, 709]","[66893, 9496, 64615, 5469736, 13487, 10163, 0]",7
1,353,"[90.0, 35.0, 9.0, 105764.0, 87886.0, 69372.0, ...","[2321, 2322, 407, 478, 80460, 69669]","[7071972, 97945, 478, 20269, 4443, 584, 57342,...",52
2,400,"[84.0, 35.0, 20.0, 3817507.0, 0.0, 94333.0, 11...","[61053, 981, 177, 20286, 20289, 10198]","[94333, 5469728, 3817507, 45459, 407, 61053, 3...",31
3,525,"[84.0, 90.0, 20.0, 709.0, 166.0, 55133.0, 75.6...","[709, 25645, 57174]","[709, 7397, 55133, 3817484, 166]",5
4,576,"[20.0, 9.0, 119.0, 177.0, 2600.0, 10757.0, 103...","[177, 102]","[2600, 177, 7064842, 60105, 52657, 0, 4443, 10...",29


In [66]:
products_id_concated = np.concatenate(train.all_prods)

In [69]:
product_ids_unique = np.unique(np.concatenate(train.all_prods))

In [74]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(products_id_concated)

LabelEncoder()

In [75]:
len(le.classes_)

734

In [76]:
products_id_concated = le.transform(products_id_concated)

In [77]:
products_id_concated

array([442, 135, 437, ...,   9,  78,  80])

In [79]:
le1 = preprocessing.LabelEncoder()
le1.fit(user_ids)
user_ids = le1.transform(user_ids)

In [81]:
X_train = sp.coo_matrix(
    (np.ones(len(user_ids)), (user_ids, products_id_concated))
)

In [82]:
X_train

<77700x734 sparse matrix of type '<class 'numpy.float64'>'
	with 2390354 stored elements in COOrdinate format>

In [85]:
model_path = 'models/lightfm_model_all_prods.pkl'

In [89]:
user_features = np.array(train.features.to_list())

In [96]:
user_features[0]

array([7.4000000e+01, 5.1000000e+01, 9.0000000e+01, 1.3487000e+04,
       6.6893000e+04, 5.4697360e+06, 1.6225572e+02, 1.4074091e+02,
       1.2900000e+02, 1.4285715e+00, 5.3452247e-01, 1.0000000e+00],
      dtype=float32)

In [92]:
train.head()

,user_id,features,target_ids,all_prods,num_targets
0,260,"[74.0, 51.0, 90.0, 13487.0, 66893.0, 5469736.0...","[66893, 158, 709]","[66893, 9496, 64615, 5469736, 13487, 10163, 0]",7
1,353,"[90.0, 35.0, 9.0, 105764.0, 87886.0, 69372.0, ...","[2321, 2322, 407, 478, 80460, 69669]","[7071972, 97945, 478, 20269, 4443, 584, 57342,...",52
2,400,"[84.0, 35.0, 20.0, 3817507.0, 0.0, 94333.0, 11...","[61053, 981, 177, 20286, 20289, 10198]","[94333, 5469728, 3817507, 45459, 407, 61053, 3...",31
3,525,"[84.0, 90.0, 20.0, 709.0, 166.0, 55133.0, 75.6...","[709, 25645, 57174]","[709, 7397, 55133, 3817484, 166]",5
4,576,"[20.0, 9.0, 119.0, 177.0, 2600.0, 10757.0, 103...","[177, 102]","[2600, 177, 7064842, 60105, 52657, 0, 4443, 10...",29


In [98]:
sp.csr_matrix(np.nan_to_num(user_features))

<77700x12 sparse matrix of type '<class 'numpy.float32'>'
	with 881013 stored elements in Compressed Sparse Row format>

In [99]:
#, user_features=sp.csr_matrix(np.nan_to_num(user_features))

In [100]:
model = LightFM(no_components=200, loss='warp', learning_rate=0.02, max_sampled=400, random_state=1, user_alpha=1e-05)

    
model.fit_partial(X_train, epochs=10, num_threads=50)
print('fitted')
model.batch_setup(
    item_chunks={0: np.arange(len(product_ids_unique))},
    n_process=50, 
)
res = model.batch_predict(chunk_id=0, user_ids=user_ids, top_k=50)
model.batch_cleanup()


fitted


In [140]:
model.batch_setup(
    item_chunks={0: np.arange(len(product_ids_unique))},
    n_process=50, 
)
res2 = model.batch_predict(chunk_id=0, user_ids=user_ids, top_k=200)
model.batch_cleanup()


In [141]:
res2[0]

(array([  0, 442, 166, 135, 170, 443, 623, 594, 347, 192, 681, 101, 169,
        646, 190, 102,  62,  51, 598, 137, 164, 595, 235,  61, 452,  60,
        100, 643, 104, 349, 599, 275,  77,  75, 246, 162, 288, 368, 660,
        345, 103,  48, 437,  86, 252, 547, 664,  33, 699, 687,  87, 630,
        607, 730, 680,  80, 316, 167, 409, 269,  54, 253, 685, 384, 373,
        254, 338, 136, 455,  81, 294, 474,  53, 168, 117, 464,  95,  76,
        161,  78, 726,  82,  73, 580, 629,  18, 513, 110,  79, 439, 291,
         83, 122, 666, 109, 324,  88, 309, 674, 650, 142, 397, 113, 377,
        413, 236, 155, 658, 422, 635, 140, 692,  34, 639, 198, 374, 108,
        306, 107, 396, 467, 408, 557, 339, 139,  92,  85, 220, 524, 606,
        652, 632, 577, 548, 106, 689,  38, 564, 676, 157, 276, 354, 398,
         94, 444, 115, 633, 141, 143,  74, 237,  97, 146, 120,  23, 145,
        552, 637, 156, 534, 264,   3, 569,  84, 114, 290, 271, 197, 160,
        105, 112, 391, 405, 385, 359, 221, 624, 272

In [182]:
res2_copy[0][0]

array([       0,    66893,    10163,     9496,    10265,    66915,
        3497570,  2586485,    45457,    13487,  5469736,     5634,
          10198,  4033462,    13481,     5642,     1598,      709,
        2833439,     9501,    10095,  2589314,    17145,     1597,
          67281,     1590,     5631,  3818498,     5650,    45459,
        2833446,    22161,     2322,     2305,    19562,    10016,
          25645,    51487,  4592908,    45256,     5649,      655,
          64615,     4443,    20285,    92368,  5044801,      285,
        6005364,  5479493,     4444,  3817487,  3302557,  9152094,
        5469728,     2325,    37061,    10189,    59094,    21827,
            876,    20286,  5479153,    57251,    55130,    20289,
          39692,     9500,    67407,     2544,    26625,    72006,
            821,    10196,     7092,    69372,     5187,     2321,
           9979,     2323,  7324996,     2575,     2271,   100849,
        3817484,      158,    76838,     6347,     2324,    65

In [142]:
for key in res2.keys():
    #print(type(res2[key][0]))
    res2[key] = list(res2[key])
    res2[key][0] = le.inverse_transform(res2[key][0])

In [145]:
import copy

In [147]:
res2_copy = copy.copy(res2)
dkeys = res2.keys()

In [152]:
import pickle

In [155]:
with open('top200.pkl', 'wb') as fp:
    pickle.dump(res2_copy, fp, pickle.HIGHEST_PROTOCOL)

In [183]:
with open('le_users.pkl', 'wb') as fp:
    pickle.dump(le1, fp, pickle.HIGHEST_PROTOCOL)

In [ ]:
uids = le1.transform(train.user_id)
itids = le.transform(items_feats.product_id)

In [156]:
train['uid_bias'] = model.user_biases[le1.transform(train.user_id)]
items_feats['itid_bias'] = model.item_biases[le.transform(items_feats.product_id)]

In [166]:
user_emb = model.user_embeddings[uids]
item_emb = model.item_embeddings[itids]

In [176]:
train['uid_embedding'] = list(user_emb)

In [178]:
train.head()

,user_id,features,target_ids,all_prods,num_targets,uid_bias,uid_embedding
0,260,"[74.0, 51.0, 90.0, 13487.0, 66893.0, 5469736.0...","[66893, 158, 709]","[66893, 9496, 64615, 5469736, 13487, 10163, 0]",7,-0.310526,"[-0.030893221, 0.022288354, 0.12546153, -0.051..."
1,353,"[90.0, 35.0, 9.0, 105764.0, 87886.0, 69372.0, ...","[2321, 2322, 407, 478, 80460, 69669]","[7071972, 97945, 478, 20269, 4443, 584, 57342,...",52,-0.832477,"[-0.07443411, -0.0976364, 0.009338922, 0.03904..."
2,400,"[84.0, 35.0, 20.0, 3817507.0, 0.0, 94333.0, 11...","[61053, 981, 177, 20286, 20289, 10198]","[94333, 5469728, 3817507, 45459, 407, 61053, 3...",31,-0.628658,"[0.01650102, -0.104130626, -0.013346228, 0.096..."
3,525,"[84.0, 90.0, 20.0, 709.0, 166.0, 55133.0, 75.6...","[709, 25645, 57174]","[709, 7397, 55133, 3817484, 166]",5,-0.257386,"[-0.009750665, -0.070757255, -0.0064769117, 0...."
4,576,"[20.0, 9.0, 119.0, 177.0, 2600.0, 10757.0, 103...","[177, 102]","[2600, 177, 7064842, 60105, 52657, 0, 4443, 10...",29,-0.626924,"[-0.058275014, -0.03513433, 0.07834611, 0.0357..."


In [179]:
train.to_csv('user_feats_lightfm.csv')

In [180]:
items_feats['itid_embedding'] = list(model.item_embeddings[itids])
items_feats.to_csv('item_feats_lightfm.csv')

In [184]:
train.to_pickle('user_feats_lightfm.pkl')

In [185]:
items_feats.to_pickle('item_feats_lightfm.pkl')